# 摘要
此專案目標在於，從華藝網站中，爬取需要論文的作者、網址、摘要等資訊，並組成資料庫，以利後續的利用。
預期利用有 : 
1. 自動化的參考文獻製作。
2. 自動化尋找過往研究使用的控制變數，從研究方法的討論出發。
3. 自動化統整過往研究的結論，作為後續推論的前提。

## Step_1
此處透過遍歷每一個頁面之資訊，蒐集所有的論文的 url ，以利後續對每一個論文網頁，發出請求。

In [ ]:
import requests, re, pandas, numpy, traceback, re, os
from bs4 import BeautifulSoup

In [ ]:
def os_path(list) : 
    os_path = os.path.dirname(os.path.dirname(os.getcwd()))
    for i in list : 
        os_path = os.path.join(os_path, i)
    return os_path

In [ ]:
# 創建錯誤資訊資料夾
os.makedirs(os_path(["程式碼", "airitilibrary_cralwer", "error", "error_crawler_step1"]), mode=0o777, exist_ok = True)

In [ ]:
# 華藝搜尋"社會學"
url = "https://www.airitilibrary.com/Search/ArticleSearch?ArticlesViewModel_SearchField=%E7%A4%BE%E6%9C%83%E5%AD%B8&ArticlesViewModel_TitleKeywordsAbstract=&ArticlesViewModel_FulltextSearchField=&ArticlesViewModel_Author=&ArticlesViewModel_JournalBookDepartment=&ArticlesViewModel_DOI=&ArticlesViewModel_ArticleArea_Taiwan=false&ArticlesViewModel_ArticleArea_ChinaHongKongMacao=false&ArticlesViewModel_ArticleArea_American=false&ArticlesViewModel_ArticleArea_Other=false&PublicationsViewModel_SearchField=&PublicationsViewModel_PublicationName=&PublicationsViewModel_ISSN=&PublicationsViewModel_PublicationUnitName=&PublicationsViewModel_DOI=&PublicationsViewModel_PublicationArea_Taiwan=false&PublicationsViewModel_PublicationArea_ChinaHongKongMacao=false&PublicationsViewModel_PublicationArea_American=false&PublicationsViewModel_PublicationArea_Other=false"

airitilibrary_get = requests.get(url)
print(airitilibrary_get) # <Response [200]>

airitilibrary_bs4 = BeautifulSoup(airitilibrary_get.text)

In [ ]:
# 推測他將每一篇文章的網址、作者 等，都儲存在 <tr>的標籤之中。
# 抓出每一個 tr 標籤，並使其形成一個 list。
td_list = airitilibrary_bs4.find_all("td", class_ = "titleB")

# 使用 for-loop 將每一篇文章的網址、作者、引用次數 等資訊，全部抓取下來。
# 每一文章的內容，都會一層一層的p標籤包裹。

td_content_list = []
for td in td_list : 
    # 抓取論文標題
    td_title = td.find_all("p")[0].text
    # print(td_title) # 義守大學《人文與社會學報》第三卷第一期編後語

    # 抓取論文網址(可連結至單篇論文之網址)
    try : 
        td_url = "https://www.airitilibrary.com/" + td.find_all("p")[0].find("a")["href"]
        # print(td_url)
    except Exception as e : 
        td_url = numpy.nan
        with open(os_path(["error", "error_crawler_step1", "url_error"])) as f : 
            # 如果是用 for-loop 去處理，這裡可能還會需要寫入哪一頁有問題需要修正。 
            f.write(traceback.format_exc() + "\n")

    # 抓取論文作者
    try : 
        td_author = td.find_all("p")[1].find("a").text
        # print(td_author) # 義守大學《人文與社會學報》編輯小組
    except : 
        td_author = numpy.nan
        with open(os_path(["error", "error_crawler_step1", "author_error"])) as f : 
            # 如果是用 for-loop 去處理，這裡可能還會需要寫入哪一頁有問題需要修正。 
            f.write(traceback.format_exc() + "\n")

    # 將內容都裝到一個 dict 中，之後輸出成 excel。
    td_content = {
        "td_title" : td_title,
        "td_url" : td_url,
    }
    td_content_list.append(td_content)

    

In [ ]:
td_content_df = pandas.DataFrame(td_content_list)
td_content_df

## Step_2
此處對每一個蒐集到的 url 發送請求，並蒐集作者、文章摘要等資訊。 

In [ ]:
# 創建錯誤資訊資料夾
os.makedirs(os_path(["程式碼", "airitilibrary_cralwer", "error", "error_crawler_step2"]), mode=0o777, exist_ok = True)


In [ ]:
# 對 td_url 之中，每一個網址發出請求。
url_content_list = []
for url in list(dict.fromkeys(td_content_df["td_url"])) :

    # 如果無法成功發送請求，則將其資訊另外蒐集起來。蒐集 title、url。
    try : 
        url_request = requests.get(url)
        url_bs4 = BeautifulSoup(url_request.text)

    except Exception as e : 
        url_title = td_content_df[td_content_df["td_url"] == url]["td_title"][0]
        error_element = {"td_title" : url_title, "td_url" : url, "error_information" : traceback.format_exc()}
        print(error_element)
        continue
    
    # 請求發送成功，接著爬取內容，並最終儲存為 CSV 檔案。
    # 作者、期刊名稱 ... ...。
    try : 
        div_detail = url_bs4.find("div", class_ = "detail")
        # print(div_detail)
    except : 
        print(traceback.format_exc())
        continue

    # 作者
    try : 
        author = numpy.nan
        p_3 = div_detail.find_all("p")[2].text
        author = re.sub("[ \r\n]", "", p_3) # 正則表達式，替換字串內容。
        author = re.sub("[；]", "、", author)

    except : 
        print(traceback.format_exc())
    
    # 論文名稱
    try : 
        paper_name = numpy.nan
        p_1 = div_detail.find_all("p")[0].text
        paper_name = re.sub("[\r\n ；]", "", p_1) # 設計社會學
    except : 
        print(traceback.format_exc())

    # 期刊名稱
    try : 
        periodicals_name = numpy.nan
        p_4 = div_detail.find_all("p")[3].text.split("；")[0]
        periodicals_name = re.sub("[\r\n ；]", "", p_4) # 設計研究
    except : 
        print(traceback.format_exc())

    # 發表日期
    try : 
        periodicals_date = numpy.nan
        p_4 = re.search("\d{4}", div_detail.find_all("p")[3].text.split("；")[1]).group(0)
        periodicals_date = re.sub("[\r\n ；]", "", p_4) # 2007
    except : 
        print(traceback.format_exc())

    # 發表卷數期數
    try : 
        periodicals_detail = numpy.nan
        p_4 = re.search("\w+期", div_detail.find_all("p")[3].text.split("；")[1]).group(0)
        periodicals_detail = re.sub("[\r\n ；]", "", p_4) # 7期
    except : 
        print(traceback.format_exc())

    # 發表頁數
    try : 
        periodicals_page = numpy.nan
        p_4 = re.search("P.*", div_detail.find_all("p")[3].text.split("；")[1]).group(0)
        periodicals_page = re.sub("[\r\n ；]", "", p_4) # P42 - 48
    except : 
        print(traceback.format_exc())

    # 摘要、引用部分
    try : 
        div_Booksainly1 = numpy.nan
        div_centerDowm = url_bs4.find("div", class_ = "centerDowm")
        div_Booksainly1 = div_centerDowm.find("div", "Booksainly1")
    except : 
        print(traceback.format_exc())
        continue

    # 摘要
    try : 
        abstract = numpy.nan
        textarea_TextArea1 = div_Booksainly1.find("textarea", "TextArea1").text
        abstract = textarea_TextArea1
    except : 
        print(traceback.format_exc())

    # 被引用次數 (有可能文章沒有被引用過)
    try : 
        citations = numpy.nan
        div_longMeshTextTitle_2 = div_Booksainly1.find_all("div", "longMeshTextTitle")[1].text
        div_longMeshTextTitle_2 = re.search("\d+", div_longMeshTextTitle_2).group(0)
        citations = re.sub("[\r\n ；]", "", div_longMeshTextTitle_2)

    except : 
        # print(traceback.format_exc())
        pass
    
    url_content = {"author" : author, "paper_name" : paper_name, "periodicals_name" : periodicals_name,
                   "periodicals_detail" : periodicals_detail, "periodicals_date" : periodicals_date, 
                   "periodicals_page" : periodicals_page, "abstract" : abstract, "citations" : citations
                   }
    url_content_list.append(url_content)



    
    

In [ ]:
url_content_df = pandas.DataFrame(url_content_list)
url_content_df

In [ ]:
# 輸出檔案
url_content_df.to_csv(os_path(["database", "華藝資料庫.csv"]), index = False)